# Analisa informações gerais dos arquivos de Twitter #

In [ ]:
#Arquivo que analisa o volume de tweets de cada série
# Bibliotecas
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from datetime import date

In [ ]:
nomes_series = ["SexEducation", "TheWitcherNetflix", "TheWitcher", "StrangerThings"]
lista_df = []

for serie in nomes_series:
    df_serie = pd.read_csv('series_dataset/'+serie+'.csv')
    df_serie["nome_serie"] = serie
    lista_df.append(df_serie)


In [ ]:
lista_df[1].head()

In [ ]:
def setDataframe(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'], format="%Y-%m-%d", errors = 'coerce')
    dataframe[dataframe["id"].duplicated()]
    

In [ ]:
for df in lista_df:
    setDataframe(df)

In [ ]:
# todos_df.groupby(["nome_serie", "date"])["date"].count()

In [ ]:
#filtro_dados = (df_TheWitcher["date"] > pd.Timestamp(2020, 1, 1)) & (df_TheWitcher["date"] <  pd.Timestamp(2020, 2, 8)

#Criar tuplas de visualização
tuplas_vis = []
for df in lista_df:

    data_final = df['date'].iloc[0] - pd.DateOffset(days=210)
    data_inicial = df['date'].iloc[-1]

    filtro_dados = (df["date"] >  data_inicial) & (df["date"] < data_final)
    y = df[filtro_dados]['date'].value_counts()
    x = df[filtro_dados]['date'].value_counts().index.tolist()

    tuplas_vis.append((x,y))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=False)

sns.lineplot(ax=axes[0,0], x = tuplas_vis[0][0], y = tuplas_vis[0][1], color="salmon")
axes[0,0].tick_params(rotation=90, axis='x')
axes[0,0].set_title(nomes_series[0])
axes[0,0].set_ylabel('Número Total de Twitts')
axes[0,0].set_xlabel('Dia')

sns.lineplot(ax=axes[0,1], x = tuplas_vis[1][0], y = tuplas_vis[1][1], color="salmon")
axes[0,1].tick_params(rotation=90, axis='x')
axes[0,1].set_title(nomes_series[1])
axes[0,1].set_ylabel('Número Total de Twitts')
axes[0,1].set_xlabel('Dia')

sns.lineplot(ax=axes[1,0], x = tuplas_vis[2][0], y = tuplas_vis[2][1], color="salmon")
axes[1,0].tick_params(rotation=90, axis='x')
axes[1,0].set_title(nomes_series[2])
axes[1,0].set_ylabel('Número Total de Twitts')
axes[1,0].set_xlabel('Dia')

sns.lineplot(ax=axes[1,1], x = tuplas_vis[3][0], y = tuplas_vis[3][1], color="salmon")
axes[1,1].tick_params(rotation=90, axis='x')
axes[1,1].set_title(nomes_series[3])
axes[1,1].set_ylabel('Número Total de Twitts')
axes[1,1].set_xlabel('Dia')
